In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

from matplotlib.cm import get_cmap
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.lines import Line2D
from statsmodels.iolib.table import SimpleTable, default_txt_fmt

In [2]:
# Lecture des données
data = pd.read_csv('data/flandreau_jobst_internationalcurrencies_data.txt', encoding='cp1252', skiprows=9, header=0, sep='\t')

In [3]:
# Commençons par créer un fichier attributes en ne gardant que les éléments par pays puis supprimant les lignes qui se répètent
attributes = data[["country_A", "gold", "colony", "debtburden", "rlong", "rshort1900", "rshort1890", "rgdp", "rgdpcap", "poldemo", "coverage"]]
# Sort the dataframe to ensure rows with colony=1 come first
attributes = attributes.sort_values(by=["country_A", "colony"], ascending=[True, False])
# Drop duplicates, keeping the first row (colony=1 will be kept if present)
attributes = attributes.drop_duplicates(subset="country_A", keep="first").reset_index(drop=True)
attributes = attributes.rename(columns={"colony": "is_colonized"})

# Récupérons aussi le nombre de colonies
nb_colonies = data.groupby("country_B")["colony"].sum().reset_index()
nb_colonies = nb_colonies.rename(columns={"colony": "has_colonies"})
attributes = nb_colonies.merge(attributes, left_on="country_B", right_on="country_A", how="left")
attributes = attributes.drop(columns=['country_A'])
attributes = attributes.rename(columns={"country_B": "country"})

In [ ]:
country_mapping = {
    "ARG": "Argentina",
    "AUS": "Australia",
    "AUH": "United Arab Emirates",
    "BEL": "Belgium",
    "BRA": "Brazil",
    "CAN": "Canada",
    "CEY": "Ceylon",
    "CHE": "Switzerland",
    "CHL": "Chile",
    "CHN": "China",
    "COL": "Colombia",
    "CUB": "Cuba",
    "DEU": "Germany",
    "DNK": "Denmark",
    "ECU": "Ecuador",
    "EGY": "Egypt",
    "ESP": "Spain",
    "FIN": "Finland",
    "FRA": "France",
    "GBR": "United Kingdom",
    "GRC": "Greece",
    "HKG": "Hong Kong",
    "ICH": "Sandwich Islands",
    "IND": "India",
    "ITA": "Italy",
    "JAV": "Java",
    "JPN": "Japan",
    "MEX": "Mexico",
    "NLD": "Netherlands",
    "NOR": "Norway",
    "NZL": "New Zealand",
    "OTT": "Ottoman Empire",
    "PER": "Peru",
    "PHL": "Philippines",
    "PRT": "Portugal",
    "PRS": "Persia",
    "ROM": "Romania",
    "RUS": "Russia",
    "SER": "Serbia",
    "SGP": "Singapore",
    "SIA": "Siam",
    "SWE": "Sweden",
    "URY": "Uruguay",
    "USA": "United States",
    "VEN": "Venezuela"}

attributes["country"] = attributes["country"].map(country_mapping)

In [ ]:
data.head(1).style.hide(axis="index")

In [ ]:
attributes.style.hide(axis="index")

# Première approche
***Description de la structure du graph ?***

Il faut d'abord bien préciser la signification des données : un lien allant de $A$ vers $B$ signifie que le pays $B$ possède un marché des changes intégrant la devise du pays $A$. En l'absence de tout autre lien, cela signifie qu'il est nécessaire de passer par le pays $B$ pour échanger la devise du pays $A$, c'est-à-dire que le pays $A$ est dépendant du pays $B$ pour ses échanges extérieurs.

_Je pense que c'est plus compliqué que cela, dès lors que le pays A a lui même un marché des changes domestique, sur lequel on peut raisonnablement penser qu'une des monnaies qui est échangée est la sienne. De ce point de vue, c'est plutôt le pays $B$ qui dépends du pays $A$. Plus un pays à un nombre important de liens pointant vers lui, plus il a accès à une diversité importante de monnaie, sans avoir besoin de recourir à un marché d'un pays tiers. Mais du coup, un pays qui n'a qu'un seul lien pointant vers lui, comme l'Autralie par exemple, n'a accès qu'à une seule devise depuis son territoire. Un pays qui n'a aucun lien pointant vers lui n'a accès à aucune devise._

In [ ]:
# Création d'un graph orienté avec la variable 1900 (possible de pondérer le graph pour utiliser les 3 ? Ou de faire avec le commerce ?)
graph_change_1900 = nx.DiGraph()

for _, row in data.iterrows():
    if row["quote1900"] == 1:
        graph_change_1900.add_edge(row["country_A"], row["country_B"])

In [ ]:
n = graph_change_1900.number_of_nodes()
L = graph_change_1900.number_of_edges()
density = L/(n*(n-1))
print(f"Nombre de sommets : {n}")
print(f"Nombre d'arêtes : {L}")
print(f"Densité : {density:.4f}")

On constate d'emblée que le réseau est très peu dense, avec seulement 218 liens sur les $45*44=1980$ possibles, ce qui correspond à une densité de 0.11 : ceci indique que la majorité des pays ne sont pas directement connectés sur le marché des changes, autrement dit, que celui-ci fonctionne de manière centralisée, ou que de nombreux pays ne participent pas aux échanges internationaux.

In [ ]:
isolates = list(nx.isolates(graph_change_1900))
print(f"Proportion de pays isolés : {len(isolates) / n:.0f}")

no_incoming = [node for node in graph_change_1900.nodes() if graph_change_1900.in_degree(node) == 0]
no_incoming_share = len(no_incoming) / n if n > 0 else 0
print(f"Proportion de pays  sans lien entrant : {100*no_incoming_share:.1f} %")

bidir_edges = sum(1 for u, v in graph_change_1900.edges() if graph_change_1900.has_edge(v, u))
total_edges = graph_change_1900.number_of_edges()
unidir_edges = total_edges - bidir_edges
country_links = unidir_edges + (bidir_edges / 2)
bidir_share = (bidir_edges / 2) / country_links
print(f"Proportion de liens qui ne sont pas réciproques : {100*(1-bidir_share):.1f} %")

print(f"La réciprocité moyenne vaut {nx.reciprocity(graph_change_1900):.3f}")

On ne trouve aucun pays isolé, ce qui écarte l'hypothèse de pays ne participant pas aux échanges internationaux. En revanche, 44,4 % des pays ne sont destinataires d'aucun lien, c'est-à-dire qu'ils doivent utiliser des marchés étrangers pour échanger leur monnaie nationale.

_A noter que c'est clairement un effet de la structure du dataset, qui est construit sous forme d'une edgelist (un peu développée, certes...). Mais un pays isolé n'apparaitrait pas ici. Bon, en même temps, on parle ici de devise, donc à un moment ou un autre ça doit puvoir être écheangable._

Si l'on s'intéresse aux liens entre pays sans considérer leur direction, mais seulement leur caractère unilatéral ou réciproque, on constate que 72,5 % des pays n'entretiennent aucun lien réciproque. _Ici je pense qu'il y a une confusion entre les edges et les nodes_

In [ ]:
def truncate_colormap(cmap_name, min_val=0.2, max_val=1.0, n=100):
    cmap = plt.get_cmap(cmap_name)
    new_cmap = LinearSegmentedColormap.from_list(
        f"{cmap_name}_trunc", cmap(np.linspace(min_val, max_val, n))
    )
    return new_cmap

trunc_oranges = truncate_colormap("Oranges", 0.2, 1.0)

# Determine edge colors
edge_colors = []
for u, v in graph_change_1900.edges():
    if graph_change_1900.has_edge(v, u):
        edge_colors.append("forestgreen")  # reciprocal
    else:
        edge_colors.append("skyblue")  # unidirectional

# Compute node colors based on total degree (in-degree + out-degree)
degrees = dict(graph_change_1900.degree())
node_colors = [degrees[node] for node in graph_change_1900.nodes()]

# Draw graph
plt.figure(figsize=(16, 8))
pos = nx.spring_layout(graph_change_1900, seed=43)

nodes = nx.draw_networkx_nodes(
    graph_change_1900,
    pos,
    node_size=600,
    node_color=node_colors,
    cmap=trunc_oranges)

edges = nx.draw_networkx_edges(
    graph_change_1900,
    pos,
    edge_color=edge_colors)

labels = nx.draw_networkx_labels(graph_change_1900, pos, font_size=9, font_color="white")

legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='forestgreen', markersize=9, label='Unidirectional'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='skyblue', markersize=9, label='Bidirectional')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.colorbar(nodes, label="Total Degree")
plt.axis("off")
plt.show()

Une représentation graphique fait apparaître :
- Un marché des changes très intégré entre l'Europe et les Etats-Unis, rassemblant une dizaine de pays (GBR, FRA, DEU, USA notamment)
- Un marché des changes secondaires en Asie, rassemblant une demi-douzaine de pays (HKG, SGP, CHN, IND notamment)
- Un relatif isolement des autres pays, qui entretiennent des liens unilatéraux avec quelques pays appartenants aux groupes précédents

(On pourrait utiliser le nombre de devises échangées plutôt que le nombre de liens pour la coloration.)

_Sauf que l'on a pas cette info dans le dataset, juste le volume du commerce..._

In [ ]:
countries = ["USA", "HKG", "EGY"]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, country_B in enumerate(countries):
    # Get the nodes related to country_B
    neighbors = list(graph_change_1900.neighbors(country_B))
    predecessors = list(graph_change_1900.predecessors(country_B))
    nodes_involved = [country_B] + neighbors + predecessors
    subgraph = graph_change_1900.subgraph(nodes_involved)
    restricted_graph = nx.DiGraph(subgraph)

    edge_colors = []
    for u, v in restricted_graph.edges():
        if restricted_graph.has_edge(v, u):
            edge_colors.append("forestgreen")  # reciprocal
        else:
            edge_colors.append("skyblue")  # unidirectional

    pos = nx.spring_layout(restricted_graph)
    ax = axes[i]
    nx.draw_networkx_nodes(restricted_graph, pos, node_size=600, node_color="chocolate", ax=ax)
    nx.draw_networkx_edges(restricted_graph, pos, edge_color=edge_colors, ax=ax)
    nx.draw_networkx_labels(restricted_graph, pos, font_size=9, font_color="white", ax=ax)
    ax.set_title(f"Graph restricted to {country_B} and its neighbors")

plt.tight_layout()
plt.show()

On peut faire apparaître les marchés intégrés en restreignant le graph aux liens réciproques ou en recherchant les cliques...

In [ ]:
graph_change_1900 = nx.DiGraph()

for _, row in data.iterrows():
    if row["quote1900"] == 1:
        graph_change_1900.add_edge(row["country_A"], row["country_B"])

# Filter for reciprocal edges
reciprocal_edges = [(u, v) for u, v in graph_change_1900.edges() if graph_change_1900.has_edge(v, u)]
# Get nodes involved in reciprocal edges
nodes_with_reciprocal_edges = set(u for u, v in reciprocal_edges) | set(v for u, v in reciprocal_edges)

plt.figure(figsize=(8, 5))
pos = nx.spring_layout(graph_change_1900, seed=42)
graph_filtered = graph_change_1900.subgraph(nodes_with_reciprocal_edges)

nx.draw_networkx_nodes(graph_filtered, pos, node_size=600, node_color="chocolate")
nx.draw_networkx_edges(graph_filtered, pos, edgelist=reciprocal_edges, edge_color="skyblue")
nx.draw_networkx_labels(graph_filtered, pos, font_size=9, font_color="white")

plt.axis("off")
plt.show()

In [ ]:
# On peut essayer de formaliser en recherchant des cliques
undirected_graph = graph_change_1900.to_undirected()
cliques = list(nx.find_cliques(undirected_graph ))
largest_cliques = sorted(cliques, key=len, reverse=True)[:1]
print(f"Plus grande clique : {largest_cliques[0]}")

In [ ]:
out_degree = graph_change_1900.out_degree()
in_degree = graph_change_1900.in_degree()

out_degree_centrality = nx.out_degree_centrality(graph_change_1900)
in_degree_centrality = nx.in_degree_centrality(graph_change_1900)
betweenness_centrality = nx.betweenness_centrality(graph_change_1900)
eigenvector_centrality = nx.eigenvector_centrality(graph_change_1900)

df = pd.DataFrame({
    "Country": [node for node, _ in out_degree],
    "Out-Degree": [degree for _, degree in out_degree],
    "In-Degree": [degree for _, degree in in_degree],
    "Out-Degree Centrality": [out_degree_centrality.get(node, 0) for node, _ in out_degree],
    "In-Degree Centrality": [in_degree_centrality.get(node, 0) for node, _ in in_degree],
    "Betweenness Centrality": [betweenness_centrality.get(node, 0) for node, _ in out_degree],
    "Eigenvector Centrality": [eigenvector_centrality.get(node, 0) for node, _ in out_degree]
})

df["In / Out ratio"] = df["In-Degree Centrality"] / df["Out-Degree Centrality"]
df = df[[*df.columns[:df.columns.get_loc("In-Degree Centrality")+1], "In / Out ratio", *df.columns[df.columns.get_loc("In-Degree Centrality")+1:].drop("In / Out ratio")]]

df_sorted = df.sort_values(by="Betweenness Centrality", ascending=False)
df_sorted.style.hide(axis="index").format({
    "Out-Degree": "{:.0f}",
    "In-Degree": "{:.0f}",
    "Out-Degree Centrality": "{:.3f}",
    "In-Degree Centrality": "{:.3f}",
    "Betweenness Centrality": "{:.3f}",
    "Average Centrality": "{:.3f}",
    "In / Out ratio": "{:.3f}",
    "Eigenvector Centrality": "{:.3f}",
})

In [ ]:
pearson_corr = df['In-Degree'].corr(df['Out-Degree'])
print(f"Corrélation entre les centralités entrantes et sortantes : {pearson_corr:.4f}")

... ou en étudiant la centralité.

La **centralité intermédiaire** fait ressortir quelques marchés constituant des points de passage obligés : GBR et DEU, ainsi dans une moindre mesure que FRA, HKG et DNK.

Le ratio entre les **centralités entrantes et sortantes**, c'est-à-dire entre le nombre de devises échangées localement par un pays, et le nombre de pays échangeant sa propre devise, permet de distinguer des situations plus contrastées.
- Les pays colonisateurs : GBR, FRA et ESP, dont la centralité sortante est très supérieure à la centralité entrante, sans doute parce qu'ils entretiennent des liens unilatéraux avec leurs colonnies et des liens bilatéraux avec les autres pays développés.
- Les pays développés : DEU, IT, USA, etc., dont les centralités entrantes et sortantes sont relativement proches, sans doute parce qu'ils entretiennent surtout des liens bilatéraux entre eux.
- Les pays périphériques / dépendants, qui n'ont aucun lien entrant.

La **centralité vectorielle** n'apporte rien par rapport aux analyses précédentes, elle oppose seulement les pays intégrés aux autres.

_Pas certain là non plus sur la centralité vectorielle, qui semble quand même écarter beaucoup le cas des USA par exemple, bien en dessous de l'Italie, de l'Autriche, la Belgique ou la Hollande..._

In [ ]:
# Jetons un oeil à la transitivité
print(f"La transitivité globale est de {nx.average_clustering(graph_change_1900):.3f}")
print(f"La transitivité moyenne est de {nx.transitivity(graph_change_1900):.3f}")

print('La transitivité locale de chaque pays est : ')
for node, coeff in sorted(nx.clustering(graph_change_1900).items(), key=lambda x: x[1]):
    print(f"{node}: {coeff:.3f}")

La transitivité n'est pas très éclairante ? _Pour le coup je ne trouve pas, non..._

## Bonus

**Est-ce qu'on voit des choses différentes en prenant une autre année ?**


In [ ]:
# Création d'un graph orienté avec la variable 18900 (possible de pondérer le graph pour utiliser les 3 ? Ou de faire avec le commerce ?)
graph_change_1890 = nx.DiGraph()

for _, row in data.iterrows():
    if row["quote1890"] == 1:
        graph_change_1890.add_edge(row["country_A"], row["country_B"])

In [ ]:
n = graph_change_1890.number_of_nodes()
L = graph_change_1890.number_of_edges()
density = L/(n*(n-1))
print(f"Nombre de sommets : {n}")
print(f"Nombre d'arêtes : {L}")
print(f"Densité : {density:.4f}")

In [ ]:
def truncate_colormap(cmap_name, min_val=0.2, max_val=1.0, n=100):
    cmap = plt.get_cmap(cmap_name)
    new_cmap = LinearSegmentedColormap.from_list(
        f"{cmap_name}_trunc", cmap(np.linspace(min_val, max_val, n))
    )
    return new_cmap

trunc_oranges = truncate_colormap("Oranges", 0.2, 1.0)

# Determine edge colors
edge_colors = []
for u, v in graph_change_1890.edges():
    if graph_change_1890.has_edge(v, u):
        edge_colors.append("forestgreen")  # reciprocal
    else:
        edge_colors.append("skyblue")  # unidirectional

# Compute node colors based on total degree (in-degree + out-degree)
degrees = dict(graph_change_1890.degree())
node_colors = [degrees[node] for node in graph_change_1890.nodes()]

# Draw graph
plt.figure(figsize=(16, 8))
pos = nx.spring_layout(graph_change_1890, seed=43)

nodes = nx.draw_networkx_nodes(
    graph_change_1890,
    pos,
    node_size=600,
    node_color=node_colors,
    cmap=trunc_oranges)

edges = nx.draw_networkx_edges(
    graph_change_1890,
    pos,
    edge_color=edge_colors)

labels = nx.draw_networkx_labels(graph_change_1890, pos, font_size=9, font_color="white")

legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='forestgreen', markersize=9, label='Unidirectional'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='skyblue', markersize=9, label='Bidirectional')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.colorbar(nodes, label="Total Degree")
plt.axis("off")
plt.show()

In [ ]:
countries = ["DEU", "HKG", "EGY"]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, country_B in enumerate(countries):
    # Get the nodes related to country_B
    neighbors = list(graph_change_1890.neighbors(country_B))
    predecessors = list(graph_change_1890.predecessors(country_B))
    nodes_involved = [country_B] + neighbors + predecessors
    subgraph = graph_change_1890.subgraph(nodes_involved)
    restricted_graph = nx.DiGraph(subgraph)

    edge_colors = []
    for u, v in restricted_graph.edges():
        if restricted_graph.has_edge(v, u):
            edge_colors.append("forestgreen")  # reciprocal
        else:
            edge_colors.append("skyblue")  # unidirectional

    pos = nx.spring_layout(restricted_graph)
    ax = axes[i]
    nx.draw_networkx_nodes(restricted_graph, pos, node_size=600, node_color="chocolate", ax=ax)
    nx.draw_networkx_edges(restricted_graph, pos, edge_color=edge_colors, ax=ax)
    nx.draw_networkx_labels(restricted_graph, pos, font_size=9, font_color="white", ax=ax)
    ax.set_title(f"Graph restricted to {country_B} and its neighbors")

plt.tight_layout()
plt.show()

In [ ]:
graph_change_1890 = nx.DiGraph()

for _, row in data.iterrows():
    if row["quote1890"] == 1:
        graph_change_1890.add_edge(row["country_A"], row["country_B"])

# Filter for reciprocal edges
reciprocal_edges = [(u, v) for u, v in graph_change_1890.edges() if graph_change_1890.has_edge(v, u)]
# Get nodes involved in reciprocal edges
nodes_with_reciprocal_edges = set(u for u, v in reciprocal_edges) | set(v for u, v in reciprocal_edges)

plt.figure(figsize=(8, 5))
pos = nx.spring_layout(graph_change_1890, seed=42)
graph_filtered = graph_change_1890.subgraph(nodes_with_reciprocal_edges)

nx.draw_networkx_nodes(graph_filtered, pos, node_size=600, node_color="chocolate")
nx.draw_networkx_edges(graph_filtered, pos, edgelist=reciprocal_edges, edge_color="skyblue")
nx.draw_networkx_labels(graph_filtered, pos, font_size=9, font_color="white")

plt.axis("off")
plt.show()

In [ ]:
out_degree = graph_change_1890.out_degree()
in_degree = graph_change_1890.in_degree()

out_degree_centrality = nx.out_degree_centrality(graph_change_1890)
in_degree_centrality = nx.in_degree_centrality(graph_change_1890)
betweenness_centrality = nx.betweenness_centrality(graph_change_1890)
eigenvector_centrality = nx.eigenvector_centrality(graph_change_1890)

df = pd.DataFrame({
    "Country": [node for node, _ in out_degree],
    "Out-Degree": [degree for _, degree in out_degree],
    "In-Degree": [degree for _, degree in in_degree],
    "Out-Degree Centrality": [out_degree_centrality.get(node, 0) for node, _ in out_degree],
    "In-Degree Centrality": [in_degree_centrality.get(node, 0) for node, _ in in_degree],
    "Betweenness Centrality": [betweenness_centrality.get(node, 0) for node, _ in out_degree],
    "Eigenvector Centrality": [eigenvector_centrality.get(node, 0) for node, _ in out_degree]
})

df["In / Out ratio"] = df["In-Degree Centrality"] / df["Out-Degree Centrality"]
df = df[[*df.columns[:df.columns.get_loc("In-Degree Centrality")+1], "In / Out ratio", *df.columns[df.columns.get_loc("In-Degree Centrality")+1:].drop("In / Out ratio")]]

df_sorted = df.sort_values(by="Betweenness Centrality", ascending=False)
df_sorted.style.hide(axis="index").format({
    "Out-Degree": "{:.0f}",
    "In-Degree": "{:.0f}",
    "Out-Degree Centrality": "{:.3f}",
    "In-Degree Centrality": "{:.3f}",
    "Betweenness Centrality": "{:.3f}",
    "Average Centrality": "{:.3f}",
    "In / Out ratio": "{:.3f}",
    "Eigenvector Centrality": "{:.3f}",
})

In [ ]:
# Création d'un graph orienté avec la variable 1910 (possible de pondérer le graph pour utiliser les 3 ? Ou de faire avec le commerce ?)
graph_change_1910 = nx.DiGraph()

for _, row in data.iterrows():
    if row["quote1910"] == 1:
        graph_change_1910.add_edge(row["country_A"], row["country_B"])

In [ ]:
n = graph_change_1910.number_of_nodes()
L = graph_change_1910.number_of_edges()
density = L/(n*(n-1))
print(f"Nombre de sommets : {n}")
print(f"Nombre d'arêtes : {L}")
print(f"Densité : {density:.4f}")

In [ ]:
def truncate_colormap(cmap_name, min_val=0.2, max_val=1.0, n=100):
    cmap = plt.get_cmap(cmap_name)
    new_cmap = LinearSegmentedColormap.from_list(
        f"{cmap_name}_trunc", cmap(np.linspace(min_val, max_val, n))
    )
    return new_cmap

trunc_oranges = truncate_colormap("Oranges", 0.2, 1.0)

# Determine edge colors
edge_colors = []
for u, v in graph_change_1910.edges():
    if graph_change_1910.has_edge(v, u):
        edge_colors.append("forestgreen")  # reciprocal
    else:
        edge_colors.append("skyblue")  # unidirectional

# Compute node colors based on total degree (in-degree + out-degree)
degrees = dict(graph_change_1910.degree())
node_colors = [degrees[node] for node in graph_change_1910.nodes()]

# Draw graph
plt.figure(figsize=(16, 8))
pos = nx.spring_layout(graph_change_1910, seed=43)

nodes = nx.draw_networkx_nodes(
    graph_change_1910,
    pos,
    node_size=600,
    node_color=node_colors,
    cmap=trunc_oranges)

edges = nx.draw_networkx_edges(
    graph_change_1910,
    pos,
    edge_color=edge_colors)

labels = nx.draw_networkx_labels(graph_change_1910, pos, font_size=9, font_color="white")

legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='forestgreen', markersize=9, label='Unidirectional'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='skyblue', markersize=9, label='Bidirectional')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.colorbar(nodes, label="Total Degree")
plt.axis("off")
plt.show()

In [ ]:
countries = ["DEU", "HKG", "EGY"]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, country_B in enumerate(countries):
    # Get the nodes related to country_B
    neighbors = list(graph_change_1910.neighbors(country_B))
    predecessors = list(graph_change_1910.predecessors(country_B))
    nodes_involved = [country_B] + neighbors + predecessors
    subgraph = graph_change_1910.subgraph(nodes_involved)
    restricted_graph = nx.DiGraph(subgraph)

    edge_colors = []
    for u, v in restricted_graph.edges():
        if restricted_graph.has_edge(v, u):
            edge_colors.append("forestgreen")  # reciprocal
        else:
            edge_colors.append("skyblue")  # unidirectional

    pos = nx.spring_layout(restricted_graph)
    ax = axes[i]
    nx.draw_networkx_nodes(restricted_graph, pos, node_size=600, node_color="chocolate", ax=ax)
    nx.draw_networkx_edges(restricted_graph, pos, edge_color=edge_colors, ax=ax)
    nx.draw_networkx_labels(restricted_graph, pos, font_size=9, font_color="white", ax=ax)
    ax.set_title(f"Graph restricted to {country_B} and its neighbors")

plt.tight_layout()
plt.show()

In [ ]:
graph_change_1910 = nx.DiGraph()

for _, row in data.iterrows():
    if row["quote1910"] == 1:
        graph_change_1910.add_edge(row["country_A"], row["country_B"])

# Filter for reciprocal edges
reciprocal_edges = [(u, v) for u, v in graph_change_1910.edges() if graph_change_1910.has_edge(v, u)]
# Get nodes involved in reciprocal edges
nodes_with_reciprocal_edges = set(u for u, v in reciprocal_edges) | set(v for u, v in reciprocal_edges)

plt.figure(figsize=(8, 5))
pos = nx.spring_layout(graph_change_1910, seed=42)
graph_filtered = graph_change_1910.subgraph(nodes_with_reciprocal_edges)

nx.draw_networkx_nodes(graph_filtered, pos, node_size=600, node_color="chocolate")
nx.draw_networkx_edges(graph_filtered, pos, edgelist=reciprocal_edges, edge_color="skyblue")
nx.draw_networkx_labels(graph_filtered, pos, font_size=9, font_color="white")

plt.axis("off")
plt.show()

In [ ]:
out_degree = graph_change_1910.out_degree()
in_degree = graph_change_1910.in_degree()

out_degree_centrality = nx.out_degree_centrality(graph_change_1910)
in_degree_centrality = nx.in_degree_centrality(graph_change_1910)
betweenness_centrality = nx.betweenness_centrality(graph_change_1910)
eigenvector_centrality = nx.eigenvector_centrality(graph_change_1910)

df = pd.DataFrame({
    "Country": [node for node, _ in out_degree],
    "Out-Degree": [degree for _, degree in out_degree],
    "In-Degree": [degree for _, degree in in_degree],
    "Out-Degree Centrality": [out_degree_centrality.get(node, 0) for node, _ in out_degree],
    "In-Degree Centrality": [in_degree_centrality.get(node, 0) for node, _ in in_degree],
    "Betweenness Centrality": [betweenness_centrality.get(node, 0) for node, _ in out_degree],
    "Eigenvector Centrality": [eigenvector_centrality.get(node, 0) for node, _ in out_degree]
})

df["In / Out ratio"] = df["In-Degree Centrality"] / df["Out-Degree Centrality"]
df = df[[*df.columns[:df.columns.get_loc("In-Degree Centrality")+1], "In / Out ratio", *df.columns[df.columns.get_loc("In-Degree Centrality")+1:].drop("In / Out ratio")]]

df_sorted = df.sort_values(by="Betweenness Centrality", ascending=False)
df_sorted.style.hide(axis="index").format({
    "Out-Degree": "{:.0f}",
    "In-Degree": "{:.0f}",
    "Out-Degree Centrality": "{:.3f}",
    "In-Degree Centrality": "{:.3f}",
    "Betweenness Centrality": "{:.3f}",
    "Average Centrality": "{:.3f}",
    "In / Out ratio": "{:.3f}",
    "Eigenvector Centrality": "{:.3f}",
})

Du coup en première approche onauqna même de vraies évolutions, avec le développement d'un marché / d'une clique secondaire en Asie, qui finit par fusionner, la place très croissante des USA (d'ailleurs faudra regarder en détail ce qu'il se passe du côte de la Betweeness Centrality, voir comment on peut expliquer cela). Mais il y a sans doute là de quoi faire un travail déjà...

# Seconde approche
***Expliquer la structure du graph avec les attributs ?***

Intéger les relations commerciales (avec `bitrade`) et financières (avec `rshort`et `rlong`) ? Par exemple, est-ce que l'absence de liens entre pays reflète la faiblesse des échanges commerciaux, ou des différentiels de taux d'intérêt ?

Pour les colonnies : aucun pays Africain dans la base

In [ ]:
# Ajoutons les attributs
attributesdata = attributes.set_index('country').to_dict('index').items()
graph_change_1900.add_nodes_from(attributesdata)

print(nx.get_node_attributes(graph_change_1900, 'gold'))

In [ ]:
# Création d'un graph des colonies 
graph_colony = nx.DiGraph()

for _, row in data.iterrows():
    if row["colony"] == 1:
        graph_colony.add_edge(row["country_A"], row["country_B"])

print(f"Nombre de sommets : {graph_colony.number_of_nodes()}")
print(f"Nombre d'arêtes : {graph_colony.number_of_edges()}")

nx.draw(graph_colony, with_labels=True, node_size=500, font_size=10)
plt.show()

# Bon par contre ça c'est vraiment un tout petit graph, on a très peu de colonies en fait. Ou plus exactement sans doute, peu de colonies qui ont un marché des changes

## Régressions

Deux approches possibles :
- Régressions **binomiales** sur les liens directionnels
- Régressions **multinomiales** sur les liens bidirectionnels

### Binomiales

In [4]:
# Convert attributes so they qualify directed edges
data['r_short_diff'] = data['rshort1900'] - data['rshort1900_B']
data['r_long_diff'] = data['rlong'] - data['rlong_B']
data['gdp_ratio'] = data['rgdp'] - data['rgdp_B']
data['gold_A_only'] = ((data['gold'] == 1) & (data['gold_B'] == 0)).astype(int)
data['gold_B_only'] = ((data['gold'] == 0) & (data['gold_B'] == 1)).astype(int)
data['gold_both'] = ((data['gold'] == 1) & (data['gold_B'] == 1)).astype(int)
data['gold_none'] = ((data['gold'] == 0) & (data['gold_B'] == 0)).astype(int)

# Replace missing values by 0 for differential interest rates
data.fillna({'r_short_diff': 0, 'r_long_diff': 0}, inplace=True)

# Define and fit the model
formula = 'quote1900 ~ dist + bitrade + colony + r_short_diff + r_long_diff + gdp_ratio + gold_A_only + gold_B_only + gold_both + gold_none'
model = smf.logit(formula, data=data).fit()

Optimization terminated successfully.
         Current function value: 0.187496
         Iterations 8


In [5]:
# Add OR to the results
table0 = model.summary().tables[0]
table1 = model.summary().tables[1]

results = table1.data
headers = results[0]
rows = results[1:]
exp_coef = [f"{np.exp(float(row[1])):.4f}" for row in rows]
headers.insert(2, 'OR')
new_rows = [row[:2] + [exp] + row[2:] for row, exp in zip(rows, exp_coef)]
new_table1 = SimpleTable(new_rows, headers, txt_fmt=default_txt_fmt)

table0_lines = table0.as_text().splitlines()
table1_lines = new_table1.as_text().splitlines()
if table1_lines[0].startswith('='):
    table1_lines = table1_lines[1:]

full_summary = "\n".join(table0_lines + table1_lines)
print(full_summary)

                           Logit Regression Results                           
Dep. Variable:              quote1900   No. Observations:                 1980
Model:                          Logit   Df Residuals:                     1970
Method:                           MLE   Df Model:                            9
Date:                Mon, 21 Apr 2025   Pseudo R-squ.:                  0.4592
Time:                        12:55:43   Log-Likelihood:                -371.24
converged:                       True   LL-Null:                       -686.52
Covariance Type:            nonrobust   LLR p-value:                5.779e-130
                coef       OR     std err      z     P>|z|    [0.025    0.975] 
-------------------------------------------------------------------------------
   Intercept     5.7301 308.0001  7.63e+06  7.51e-07  1.000  -1.5e+07   1.5e+07
        dist    -1.2533   0.2856     0.107   -11.731  0.000    -1.463    -1.044
     bitrade  7.289e-06   1.0000  8.61e-07     8

Interprétation : outcome = 1 si la monnaie du pays B est échangée localement dans le pays A, 0 sinon

Variables significatives :
- Effet négatif
    - Distance entre A et B => plus A est éloigné de B et moins il est susceptible d'échanger localement sa monnaie
    - Ratio des PIB => plus le PIB de A est élevé relativement à celui de B, moins A est susceptible d'échanger localement la monnaie de B (la variable est égale à log(PIB_A/PIB_B))
- Effet positif
    - Statut colonial => A est plus susceptible d'échanger localement la monnaie de B s'il est une colonie de B
    - Différentiel de taux d'intérêt à CT => A est plus susceptible d'échanger localement la monnaie de B si son taux d'intérêt à CT est supérieur à celui de B
- Effet neutre / pas d'effet
    - Echanges commerciaux, avec un OR quasi-égal à 1
    - Convertibilité en or

### Multinomiales
#### Update the dataset to reflect directed attributes of the edges

In [6]:
data['pair_key'] = data.apply(lambda row: tuple(sorted([row['country_A'], row['country_B']])), axis=1)

def compute_combined_features(group):
    a, b = group['pair_key'].iloc[0]

    a_to_b = group[(group['country_A'] == a) & (group['country_B'] == b)]
    b_to_a = group[(group['country_A'] == b) & (group['country_B'] == a)]

    a_to_b_quote = a_to_b['quote1900'].iloc[0] if not a_to_b.empty else 0
    b_to_a_quote = b_to_a['quote1900'].iloc[0] if not b_to_a.empty else 0

    if a_to_b_quote == 1 and b_to_a_quote == 1:
        currency_market = 3  # or 2 to distinguish only between unidirectional and bidirectional links, their direction not withstanding
    elif a_to_b_quote == 1:
        currency_market = 1
    elif b_to_a_quote == 1:
        currency_market = 2  # or 1 for the same reason
    else:
        currency_market = 0

    a_to_b_colony = a_to_b['colony'].iloc[0] if not a_to_b.empty else 0
    b_to_a_colony = b_to_a['colony'].iloc[0] if not b_to_a.empty else 0

    if a_to_b_colony == 1 and b_to_a_colony == 1:
        colony_status = 3
    elif a_to_b_colony == 1:
        colony_status = 1
    elif b_to_a_colony == 1:
        colony_status = 2
    else:
        colony_status = 0

    a_to_b_gold = a_to_b['gold'].iloc[0] if not a_to_b.empty else 0
    b_to_a_gold = b_to_a['gold'].iloc[0] if not b_to_a.empty else 0

    if a_to_b_gold == 1 and b_to_a_gold == 1:
        gold_status = 3
    elif a_to_b_gold == 1:
        gold_status = 1
    elif b_to_a_gold == 1:
        gold_status = 2
    else:
        gold_status = 0
    
    dist_value = group['dist'].iloc[0]
    bitrade_value = group['bitrade'].iloc[0]

    # Retain all extra variables from the A->B row if available
    extra_columns = ['r_short_diff', 'r_long_diff', 'gdp_ratio']
    
    extras = {}
    if not a_to_b.empty:
        for col in extra_columns:
            extras[col] = a_to_b[col].iloc[0]
    else:
        # If no A->B row exists, fill with NaN
        for col in extra_columns:
            extras[col] = pd.NA

    return pd.Series({
        'country_A': a,
        'country_B': b,
        'currency_market': currency_market,
        'colony': colony_status,
        'gold': gold_status,
        'dist': dist_value,
        'bitrade': bitrade_value,
        **extras
    })

# Apply groupby
synthetic_data = data.groupby('pair_key').apply(compute_combined_features).reset_index(drop=True)
synthetic_data['currency_market'] = synthetic_data['currency_market'].astype('category')
synthetic_data['colony'] = synthetic_data['colony'].astype('category')
synthetic_data['gold'] = synthetic_data['gold'].astype('category')

/var/folders/39/44r9ct3n4hj3sdf26gj5wgp40000gn/T/ipykernel_23069/2127431410.py:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  synthetic_data = data.groupby('pair_key').apply(compute_combined_features).reset_index(drop=True)


#### Consistency check for a given row

In [7]:
synthetic_data[(synthetic_data['currency_market']==1) & (synthetic_data['colony']==1)].head(1).style.hide(axis="index")

country_A,country_B,currency_market,colony,gold,dist,bitrade,r_short_diff,r_long_diff,gdp_ratio
AUS,GBR,1,1,3,9.735524,1134737.000000,1.560000,0.570000,-2.510750


In [8]:
# Values in the original df are identical
data[(data['country_A']=='AUS') & (data['country_B']=='GBR')].style.hide(axis="index")

country_A,country_B,quote1890,quote1900,quote1910,colony,dist,bitrade,gold,debtburden,rlong,rshort1900,rshort1890,rgdp,rgdpcap,poldemo,coverage,gold_B,debtburden_B,rlong_B,rshort1900_B,rshort1890_B,rgdp_B,rgdpcap_B,poldemo_B,r_short_diff,r_long_diff,gdp_ratio,gold_A_only,gold_B_only,gold_both,gold_none,pair_key
AUS,GBR,1,1,1,1,9.735524,1134737.000000,1,6.065920,3.120000,5.500000,7.000000,16.524400,8.297290,6.370000,0.000000,1,5.028125,2.550000,3.940000,4.520000,19.035150,8.410050,3.680000,1.560000,0.570000,-2.510750,0,0,1,0,"('AUS', 'GBR')"


In [9]:
# The symetric pair does not exist
synthetic_data[(synthetic_data['country_A']=='GBR') & (synthetic_data['country_B']=='AUS')]

,country_A,country_B,currency_market,colony,gold,dist,bitrade,r_short_diff,r_long_diff,gdp_ratio


#### Fit the model
Statsmodels does not handle categorical variables natively...

In [10]:
# Create dummies for categorical variables
## Colony, the reference being no country of the pair is a colony of the other
## Must be excluded from the model, which does not fit otherwise. Indeed, colony is ≠ 0 for only 12 observations out of 990.
# colony_dummies = pd.get_dummies(synthetic_data['colony'], prefix='colony', drop_first=True).astype(int)
## Gold, the reference being no country of the pair adheres to the gold standard
gold_dummies = pd.get_dummies(synthetic_data['gold'], prefix='gold', drop_first=True).astype(int)

# Build the dataset
continuous_data = synthetic_data[['dist', 'bitrade', 'r_short_diff', 'r_long_diff', 'gdp_ratio']]
independent_variables = pd.concat([continuous_data, gold_dummies], axis=1)
dependent_variable = synthetic_data['currency_market']

# Define and fit the model
model = sm.MNLogit(dependent_variable, independent_variables).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 0.303911
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:        currency_market   No. Observations:                  990
Model:                        MNLogit   Df Residuals:                      966
Method:                           MLE   Df Model:                           21
Date:                Mon, 21 Apr 2025   Pseudo R-squ.:                  0.5307
Time:                        12:56:03   Log-Likelihood:                -300.87
converged:                       True   LL-Null:                       -641.05
Covariance Type:            nonrobust   LLR p-value:                1.870e-130
=====================================================================================
currency_market=1       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
dist                 -0.7354      0.094     -7.814      0.000      -0.920      -0.551
bitrade            3.491e-05   3.59e-06      9.716      0.000    2.79e-05    4.19e-05
r_short_diff          0.6460      0.137      4.718      0.000       0.378       0.914
r_long_diff          -0.0998      0.108     -0.928      0.353      -0.311       0.111
gdp_ratio            -0.6526      0.123     -5.285      0.000      -0.895      -0.411
gold_1                1.4328      0.858      1.669      0.095      -0.249       3.115
gold_2                0.6499      0.818      0.794      0.427      -0.954       2.254
gold_3                0.5505      0.752      0.732      0.464      -0.923       2.024
-------------------------------------------------------------------------------------
currency_market=2       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
dist                 -0.5153      0.056     -9.187      0.000      -0.625      -0.405
bitrade             3.69e-05   3.58e-06     10.315      0.000    2.99e-05    4.39e-05
r_short_diff         -0.3137      0.091     -3.431      0.001      -0.493      -0.135
r_long_diff          -0.1129      0.063     -1.779      0.075      -0.237       0.011
gdp_ratio             0.4404      0.086      5.122      0.000       0.272       0.609
gold_1                0.0485      0.523      0.093      0.926      -0.976       1.073
gold_2               -3.0127      1.249     -2.412      0.016      -5.460      -0.565
gold_3               -0.1427      0.497     -0.287      0.774      -1.117       0.832
-------------------------------------------------------------------------------------
currency_market=3       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
dist                 -0.9136      0.153     -5.972      0.000      -1.213      -0.614
bitrade            3.773e-05   3.59e-06     10.505      0.000    3.07e-05    4.48e-05
r_short_diff         -0.0338      0.139     -0.243      0.808      -0.306       0.239
r_long_diff          -0.0042      0.125     -0.034      0.973      -0.249       0.240
gdp_ratio             0.2235      0.113      1.978      0.048       0.002       0.445
gold_1                2.9069      1.323      2.197      0.028       0.314       5.500
gold_2                1.7463      1.417      1.233      0.218      -1.030       4.523
gold_3                2.8715      1.191      2.411      0.016       0.537       5.206
=====================================================================================
"""

Interprétation : outcome = 1 si la monnaie du pays B est échangée localement dans le pays A, 2 pour l'inverse, 3 si A et B échangent localement leur monnaie, 0 si aucun des deux n'échange localement la monnaie de l'autre (modalité de référence)

On retrouve à peu près les mêmes variables significatives avec des effets de même signe.
- L'effet de la convertibilité en or devient significatif à 10 % lorsque la monnaie du pays échangeant localement celle de l'autre est convertible en or (gold = 1 si seule la monnaie du pays A est convertible, 2 si seule la monnaie du pays B est convertible, 3 lorsque les deux monnaies sont convertibles, 0 sinon).
- L'effet du différentiel de taux d'intérêt à CT disparaît sur la probabilité que deux pays échangent mutuellement leur monnaie.


*Il est aussi possible de fusionner les modalités 1 et 2 pour comparer seulement l'absence de lien à l'existence de liens unilatéraux ou bilatéraux, sans considérer leur direction. Les différentiels de taux d'intérêt et de PIB ne sont plus significatifs : reste seulement la distance, le volume du commerce (mais avec un coefficient toujours nul) et la convertibilité en or (avec un effet massif, OR de l'ordre de 20).*